<a href="https://colab.research.google.com/github/kyrajeep/DL_Projects/blob/master/detect_mask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from bs4 import BeautifulSoup
import torchvision
from torchvision import transforms, datasets, models
import torch
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from PIL import Image
import matplotlib.pyplot as plt
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
import matplotlib.patches as patches
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report


In [3]:
from transformers import AutoModelForImageClassification, ViTImageProcessor
import os

In [4]:
!pip install kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"kjpark5","key":"db53a93307c0b9193869bb4906bddf90"}'}

In [5]:
# use the Kaggle API to directly download data
!mkdir ~/.kaggle
#files.upload()
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
#!kaggle datasets list


In [6]:
!kaggle datasets download -d andrewmvd/face-mask-detection
!unzip face-mask-detection.zip


 99% 392M/398M [00:03<00:00, 144MB/s]
100% 398M/398M [00:03<00:00, 136MB/s]
Archive:  face-mask-detection.zip
  inflating: annotations/maksssksksss0.xml  
  inflating: annotations/maksssksksss1.xml  
  inflating: annotations/maksssksksss10.xml  
  inflating: annotations/maksssksksss100.xml  
  inflating: annotations/maksssksksss101.xml  
  inflating: annotations/maksssksksss102.xml  
  inflating: annotations/maksssksksss103.xml  
  inflating: annotations/maksssksksss104.xml  
  inflating: annotations/maksssksksss105.xml  
  inflating: annotations/maksssksksss106.xml  
  inflating: annotations/maksssksksss107.xml  
  inflating: annotations/maksssksksss108.xml  
  inflating: annotations/maksssksksss109.xml  
  inflating: annotations/maksssksksss11.xml  
  inflating: annotations/maksssksksss110.xml  
  inflating: annotations/maksssksksss111.xml  
  inflating: annotations/maksssksksss112.xml  
  inflating: annotations/maksssksksss113.xml  
  inflating: annotations/maksssksksss114.xml  
  i

In [7]:
!ls

annotations  face-mask-detection.zip  images  sample_data


In [8]:
images = list(sorted(os.listdir("images/")))
labels = list(sorted(os.listdir("annotations/")))

In [9]:
def generate_box(obj):

    xmin = int(obj.find('xmin').text)
    ymin = int(obj.find('ymin').text)
    xmax = int(obj.find('xmax').text)
    ymax = int(obj.find('ymax').text)

    return [xmin, ymin, xmax, ymax]

def generate_label(obj):
    if obj.find('name').text == "with_mask":
        return 1
    elif obj.find('name').text == "mask_weared_incorrect":
        return 2
    return 0

def generate_target(image_id, file):
    with open(file) as f:
        data = f.read()
        soup = BeautifulSoup(data, 'xml')
        objects = soup.find_all('object')

        num_objs = len(objects)

        # Bounding boxes for objects
        # In coco format, bbox = [xmin, ymin, width, height]
        # In pytorch, the input should be [xmin, ymin, xmax, ymax]
        boxes = []
        labels = []
        for i in objects:
            boxes.append(generate_box(i))
            labels.append(generate_label(i))
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # Labels (In my case, I only one class: target class or background)
        labels = torch.as_tensor(labels, dtype=torch.int64)
        # Tensorise img_id
        img_id = torch.tensor([image_id])
        # Annotation is in dictionary format
        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["image_id"] = img_id

        return target

In [10]:
class MaskDataset(object):
    def __init__(self, transforms):
        self.transforms = transforms
        # load all image files, sorting them to
        # ensure that they are aligned
        self.imgs = list(sorted(os.listdir("images/")))
#         self.labels = list(sorted(os.listdir("/kaggle/input/face-mask-detection/annotations/")))

    def __getitem__(self, idx):
        # load images ad masks
        file_image = 'maksssksksss'+ str(idx) + '.png'
        file_label = 'maksssksksss'+ str(idx) + '.xml'
        img_path = os.path.join("images/", file_image)
        label_path = os.path.join("annotations/", file_label)
        img = Image.open(img_path).convert("RGB")
        #Generate Label
        target = generate_target(idx, label_path)

        if self.transforms is not None:
            img = self.transforms(img)

        return img, target

    def __len__(self):
        return len(self.imgs)



In [11]:
data_transform = transforms.Compose([
        transforms.ToTensor(),
    ])


In [12]:
#batch data with different sizes without manual padding
#https://stackoverflow.com/questions/65279115/how-to-use-collate-fn-with-dataloaders
def collate_fn(batch):
    return tuple(zip(*batch))

dataset = MaskDataset(data_transform)

In [13]:
train_size=int(len(dataset)*0.7)
test_size=len(dataset)-train_size
print('Length of dataset is', len(dataset), '\nLength of training set is :',train_size,'\nLength of test set is :', test_size)


Length of dataset is 853 
Length of training set is : 597 
Length of test set is : 256


In [14]:
trainset, testset=torch.utils.data.random_split(dataset,[train_size,test_size])

data_loader = torch.utils.data.DataLoader(
 trainset, batch_size=4, collate_fn=collate_fn)


In [15]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
for imgs, annotations in data_loader:
    imgs = list(img.to(device) for img in imgs)
    img_size = imgs[0].size()
    annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
    print(annotations)
    break


[{'boxes': tensor([[ 98., 267., 194., 383.]], device='cuda:0'), 'labels': tensor([1], device='cuda:0'), 'image_id': tensor([10], device='cuda:0')}, {'boxes': tensor([[209.,  73., 247., 113.],
        [287.,  60., 328., 115.],
        [354., 109., 400., 191.],
        [156.,  20., 173.,  37.]], device='cuda:0'), 'labels': tensor([1, 2, 1, 0], device='cuda:0'), 'image_id': tensor([387], device='cuda:0')}, {'boxes': tensor([[ 37.,  60., 112., 137.],
        [145., 116., 229., 209.],
        [268.,  16., 380., 135.]], device='cuda:0'), 'labels': tensor([1, 1, 0], device='cuda:0'), 'image_id': tensor([83], device='cuda:0')}, {'boxes': tensor([[ 12.,  37.,  54.,  79.],
        [147.,  47., 197., 103.],
        [292.,  55., 315.,  80.],
        [381.,  56., 400.,  77.]], device='cuda:0'), 'labels': tensor([1, 1, 1, 1], device='cuda:0'), 'image_id': tensor([685], device='cuda:0')}]


In [16]:
from transformers import DetrImageProcessor, DetrForObjectDetection
import torch
from PIL import Image
import requests

#url = "http://images.cocodataset.org/val2017/000000039769.jpg"
#image = Image.open(requests.get(url, stream=True).raw)

# use Detr model from huggingface
# you can specify the revision tag if you don't want the timm dependency
processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50", revision="no_timm")
model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50", revision="no_timm")

inputs = processor(images=imgs, return_tensors="pt", do_rescale=False)
outputs = model(**inputs)




/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/401 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/6.60k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/167M [00:00<?, ?B/s]

In [22]:
import torch.nn as nn
from transformers import AutoModel, AutoConfig
# Modify the model architecture to enable finetuning

# Load pre-trained DeTR model
model_name = "facebook/detr-resnet-50"

config = AutoConfig.from_pretrained(model_name)
#print(config)
model = AutoModel.from_pretrained(model_name, config=config)

# Replace classification head
num_classes = 200  # Assuming 3 classes: person, bag, glasses
model.class_embed = nn.Linear(model.config.d_model, num_classes)
model.num_classes = num_classes  # Update number of classes in the config

# Freeze all layers except the final layers
for param in model.parameters():
    param.requires_grad = False

# Make final layers trainable
for param in model.class_embed.parameters():
    param.requires_grad = True

# Optionally, you may also make other final layers trainable, if applicable
# For example, if you have additional layers after class_embed, you can make them trainable:
# for param in model.additional_final_layers.parameters():
#     param.requires_grad = True

# You can verify which layers are trainable
for name, param in model.named_parameters():
    print(f"Layer: {name}, Trainable: {param.requires_grad}")

Layer: backbone.conv_encoder.model.conv1.weight, Trainable: False
Layer: backbone.conv_encoder.model.layer1.0.conv1.weight, Trainable: False
Layer: backbone.conv_encoder.model.layer1.0.conv2.weight, Trainable: False
Layer: backbone.conv_encoder.model.layer1.0.conv3.weight, Trainable: False
Layer: backbone.conv_encoder.model.layer1.0.downsample.0.weight, Trainable: False
Layer: backbone.conv_encoder.model.layer1.1.conv1.weight, Trainable: False
Layer: backbone.conv_encoder.model.layer1.1.conv2.weight, Trainable: False
Layer: backbone.conv_encoder.model.layer1.1.conv3.weight, Trainable: False
Layer: backbone.conv_encoder.model.layer1.2.conv1.weight, Trainable: False
Layer: backbone.conv_encoder.model.layer1.2.conv2.weight, Trainable: False
Layer: backbone.conv_encoder.model.layer1.2.conv3.weight, Trainable: False
Layer: backbone.conv_encoder.model.layer2.0.conv1.weight, Trainable: False
Layer: backbone.conv_encoder.model.layer2.0.conv2.weight, Trainable: False
Layer: backbone.conv_encode

In [23]:
# convert outputs (bounding boxes and class logits) to COCO API
# let's only keep detections with score > 0.9
target_sizes = [img.shape[-2:] for img in imgs]
results = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.9)[0]

for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
    box = [round(i, 2) for i in box.tolist()]
    print(
            f"Detected {model.config.id2label[label.item()]} with confidence "
            f"{round(score.item(), 3)} at location {box}"
    )

Detected cell phone with confidence 0.966 at location [81.9, 322.27, 98.83, 341.31]
Detected person with confidence 0.972 at location [4.23, 224.96, 252.34, 399.77]


In [24]:
def calculate_iou(box1, box2):
    """
    Calculate IoU (Intersection over Union) between two bounding boxes.

    Arguments:
    box1: List or tuple containing [xmin, ymin, xmax, ymax] of the first box.
    box2: List or tuple containing [xmin, ymin, xmax, ymax] of the second box.

    Returns:
    IoU: Intersection over Union between the two boxes.
    """
    # Calculate intersection (area of overlap)
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])

    intersection = max(0, x2 - x1) * max(0, y2 - y1)

    # Calculate union (area of combined bounding boxes)
    area_box1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area_box2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union = area_box1 + area_box2 - intersection

    # Calculate IoU
    iou = intersection / union if union > 0 else 0

    return iou


In [ ]:
# Creating a data loader for the train set
train_loader = torch.utils.data.DataLoader(
    trainset, batch_size=4, collate_fn=collate_fn
)


In [27]:
from torch.optim import Adam
import torch.nn.functional as F
from tqdm import tqdm


# Define optimizer and learning rate scheduler
optimizer = Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=0.001, gamma=0.001)
num_epochs = 10
# Training loop
model.train()
for epoch in range(num_epochs):
    total_loss = 0.0
    for images, annotations in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False):
        optimizer.zero_grad()
        images = [img.to(device) for img in images]
        annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
        inputs = processor(images=images, return_tensors="pt", do_rescale=False)
        outputs = model(**inputs)

        # Compute loss
        loss = calculate_iou(outputs, annotations)  # Define your custom loss function
        total_loss += loss.item()

        # Backpropagation
        loss.backward()
        optimizer.step()

    # Update learning rate
    scheduler.step()

    # Compute average loss for the epoch
    avg_loss = total_loss / len(train_loader)

    # Print average loss for the epoch
    print(f"Epoch {epoch+1}/{num_epochs}, Avg. Loss: {avg_loss:.4f}")

# Save the trained model
model.save_pretrained()


TypeError: '>' not supported between instances of 'dict' and 'Tensor'

In [26]:


# Device configuration
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
threshold = 0.5
# Ground truth and predicted labels
gt_labels_all = []
pred_labels_all = []

# Evaluation loop
for imgs, annotations in train_loader:
    imgs = [img.to(device) for img in imgs]
    img_size = imgs[0].size()
    annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
    #model.eval()?
    # Processing the images through the model
    inputs = processor(images=imgs, return_tensors="pt", do_rescale=False)
    outputs = model(**inputs)



    # Post-processing object detection results
    target_sizes = [img.shape[-2:] for img in imgs]
    results = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.9)[0]

    # Model predictions
    pred_boxes = results["boxes"].tolist()
    pred_labels = results["labels"].tolist()

    # Ground truth annotations
    gt_boxes = [ann["boxes"].tolist() for ann in annotations]
    gt_labels = [ann["labels"].tolist() for ann in annotations]

    # Compare predictions with ground truth
    for pred_box, pred_label, gt_box, gt_label in zip(pred_boxes, pred_labels, gt_boxes, gt_labels):
        # Compare each predicted box with ground truth boxes for the same image
        for i, gt_box_i in enumerate(gt_box):
            iou = calculate_iou(pred_box, gt_box_i)  # Calculate Intersection over Union
            if iou > threshold:  # You need to define a threshold to consider a detection as correct
                pred_label_name = model.config.id2label[pred_label]
                gt_label_name = model.config.id2label[gt_label[i]]
                print(f"Prediction: {pred_box} - Label: {pred_label_name}")
                print(f"Ground Truth: {gt_box_i} - Label: {gt_label_name}")
                print(f"IoU: {iou}")
                gt_labels_all.append(gt_label_name)
                pred_labels_all.append(pred_label_name)


# with no fine-tuning and threshold at 0.6, no image passed the threshold.
# TODO: finetune?

AttributeError: 'DetrModelOutput' object has no attribute 'logits'

In [ ]:
# Creating a data loader for the test set
test_loader = torch.utils.data.DataLoader(
    testset, batch_size=4, collate_fn=collate_fn
)
